In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

In [ ]:
# common genes between datasets
df = pd.read_csv("../../data/GTEx/bulk-gex_v8_rna-seq_tpms-by-tissue_gene_tpm_2017-06-05_v8_cells_ebv-transformed_lymphocytes.gct.gz", sep="\t", skiprows=2)
gtex_genes = df["Name"].values
gtex_genes = [ii.split(".")[0] for ii in gtex_genes]
df['Name'] = gtex_genes
gtex_genes = set(gtex_genes)

phastcons = pd.read_csv("../../data/conservation/HS_phyloP.tab", sep="\t", header=None)
phastcons_genes = set(phastcons[0].unique())

genes = set.intersection(gtex_genes, phastcons_genes)

In [ ]:
# X
df = df[df['Name'].isin(genes)].copy()
del df['id'], df['Description']
df = df.set_index("Name")
mean_qt = df.mean(axis=1).quantile(0.95)
std_qt = df.std(axis=1).quantile(0.95)
df = df[(df.std(axis=1) > std_qt) & (df.mean(axis=1) > mean_qt)]

df = df.sort_index()
df = df.T.sort_index().T

df.index.name = None
X = df.copy()
X.to_csv("../../result/input_GTEx_phyloP/X", sep="\t")

In [ ]:
Y = phastcons.loc[phastcons[0].isin(X.index.values), [0,5]]
Y.columns = ['gene', 'conservation']
Y = Y.sort_values("gene")
Y.to_csv("../../result/input_GTEx_phyloP/Y", sep="\t", index=False)